<a href="https://colab.research.google.com/github/pbluc3m/collab-R/blob/main/procesar_tfe_2025_02_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# PARTE 0: cojo los ficheros de Olivia y Teresa y los uno para lo que me interesa

import pandas as pd

input_teresa = 'JASPER_TFE_EPS_23-24.xlsx'
input_olivia = 'JASPER_TFF_COLME_23-24.xls'

df_teresa = pd.read_excel(input_teresa)
df_olivia = pd.read_excel(input_olivia)

columnas = ["NIA", "ALUMNO", "COD.CENTRO", "CENTRO", "COD.EST", "ESTUDIOS", "COD.PLAN", "PLAN", "COD.ASIG", "ASIGNATURA", "COD.OFERTA", "TITULO", "IDIOMA", "NOTA", "CALIF", "SUPERADA", "FECHA CALIF.", "TUTOR", "DEPARTAMENTO_TUTOR", "TUTOR_EXTERNO"]
df_teresa_c = df_teresa.loc[:, df_teresa.columns.intersection(columnas)]
df_olivia_c = df_olivia.loc[:, df_olivia.columns.intersection(columnas)]

df_raw = pd.concat([df_teresa_c, df_olivia_c], ignore_index=True)

# Total a procesar
total_rows = df_raw.shape[0]
print(f"Total TFEs: {total_rows}")


Total TFEs: 1350


In [26]:
# FILTRAMOS SUPERADAS que no sean ADE y sin DUPLICADOS del doble grado

# Vamos a quedarnos con los superados
df_sup = df_raw[df_raw["SUPERADA"].str.strip().str.upper() == "S"]

# Vamos a quitar los de ADE
df = df[df["ASIGNATURA"] != "Trabajo Fin de Grado (ADE)"]

# Quito los TFE que sean la misma oferta (problema con dobles grados)
#df = df.drop_duplicates(subset='COD.OFERTA', keep='first')
df = df[df['FECHA CALIF.'].notna()]

total_rows = df.shape[0]
print(f"Total TFEs: {total_rows}")

Total TFEs: 1284


In [23]:
# Estadisticas vs. Marina 2025-02-24

# PARTE 1: BASIC STATS

# Por centro
centro = df.groupby("CENTRO").size()
print(centro)

# Por estudios
estudio_counts = df.groupby("ESTUDIOS").size()
print(estudio_counts)

CENTRO
Escuela Politécnica Superior (Colmenarejo)             58
Escuela Politécnica Superior (Leganés)                952
Escuela Politécnica Superior - Centro de Postgrado    274
dtype: int64
ESTUDIOS
Alumnos de Programas de Intercambio                                                      1
Doble Grado en Ingeniería Física e Ingeniería en Tecnologías Industriales                1
Doble Grado en Ingeniería Informática y Administración de Empresas                      28
Grado en Ciencia e Ingeniería de Datos                                                  48
Grado en Ciencias                                                                        1
Grado en Ingeniería Aeroespacial                                                        89
Grado en Ingeniería Biomédica                                                           53
Grado en Ingeniería Electrónica Industrial y Automática                                 94
Grado en Ingeniería Eléctrica                                        

In [ ]:
# Vamos a arreglar gente sin departamento
# Y corregir nombres

# Asignar tutores a departamento
df.loc[df['TUTOR'] == "GONZALEZ CABAÑAS, JOSE", 'DEPARTAMENTO_TUTOR'] = "Departamento de Ingeniería Telemática"
df.loc[df['TUTOR'] == "GOMEZ MARTIN, ADRIAN", 'DEPARTAMENTO_TUTOR'] = "Departamento de Bioingeniería"

# 2025-02-18 Jero (TSC) detecta uno que estaba en Informática
df.loc[df['TUTOR'] == "MARTINEZ CORTES, TOMAS", 'DEPARTAMENTO_TUTOR'] = "Departamento de Teoría de la Señal y Comunicaciones"

df['PESO'] = 1.0
# 2025-02-20 Dolores (Automática) me avisa de co-tutela con Informática
df.loc[df['ALUMNO'] == 'RAMOS GARCIA, MIGUEL', 'PESO'] = 0.5
row_to_duplicate = df[df['ALUMNO'] == 'RAMOS GARCIA, MIGUEL'].copy()
row_to_duplicate['TUTOR'] = 'CASTRO GONZALEZ, ALVARO'
row_to_duplicate['DEPARTAMENTO_TUTOR'] = 'Departamento de Ingeniería de Sistemas y Automática'
df = pd.concat([df, row_to_duplicate], ignore_index=True)

# Para doble grado Informatica ADE,
df['ASIGNATURA'] = df['ASIGNATURA'].replace(
    "Trabajo Fin de Grado (Informática)",
    "Trabajo Fin de Grado"
)
# La tilde
df['ASIGNATURA'] = df['ASIGNATURA'].replace(
    "Trabajo Fin de Master",
    "Trabajo Fin de Máster"
)
# La 'de'
df['ASIGNATURA'] = df['ASIGNATURA'].replace(
    "Trabajo de Fin de Máster",
    "Trabajo Fin de Máster"
)

# Confirmamos el total de TFMs y TFGs
asignatura = df.groupby("ASIGNATURA").size()
print(asignatura)

ASIGNATURA
Trabajo Fin de Grado     1010
Trabajo Fin de Máster     275
dtype: int64


In [ ]:
# PARTE 1: BASIC STATS

# Por centro
centro = df.groupby("CENTRO").size()
print(centro)

# Por departamento
departamento = df.groupby("DEPARTAMENTO_TUTOR").size()
print(departamento)

estudio_counts = df.groupby("ESTUDIOS").size()
# Display the counts
print(estudio_counts)

CENTRO
Escuela Politécnica Superior (Colmenarejo)             58
Escuela Politécnica Superior (Leganés)                952
Escuela Politécnica Superior - Centro de Postgrado    274
dtype: int64
DEPARTAMENTO_TUTOR
Departamento de Bioingeniería                                               41
Departamento de Ciencia e Ingeniería de Materiales e Ingenieria Química     24
Departamento de Economía de la Empresa                                       1
Departamento de Estadística                                                 17
Departamento de Física                                                      10
Departamento de Informática                                                216
Departamento de Ingeniería Aeroespacial                                    110
Departamento de Ingeniería Eléctrica                                        81
Departamento de Ingeniería Mecánica                                        141
Departamento de Ingeniería Telemática                                      1

In [ ]:
# PARTE 2: Generar archivo por departamento
import unicodedata

# Quitamos los acentos

def remove_accents(text):
    if isinstance(text, str):
        return "".join(c for c in unicodedata.normalize("NFD", text) if unicodedata.category(c) != "Mn")
    return text

df["DEPARTAMENTO_TUTOR"] = df["DEPARTAMENTO_TUTOR"].apply(remove_accents)

# Estadisticas
total_rows = df.shape[0]
print(f"Total number of rows: {total_rows}")

filtered_df = df[df["DEPARTAMENTO_TUTOR"].str.startswith("Departamento de", na=False)]
total_rows = filtered_df.shape[0]
print(f"Total number of rows where Departamento de: {total_rows}")


Total number of rows: 1305
Total number of rows where Departamento de: 1305


In [ ]:
# Quito lo de "Departamento de"

df["DEPARTAMENTO_TUTOR"] = df["DEPARTAMENTO_TUTOR"].str.replace("Departamento de ", "", regex=False)
# Por departamento
departamento = df.groupby("DEPARTAMENTO_TUTOR").size()
print(departamento)

DEPARTAMENTO_TUTOR
Bioingenieria                                               41
Ciencia e Ingenieria de Materiales e Ingenieria Quimica     24
Economia de la Empresa                                       1
Estadistica                                                 17
Fisica                                                      10
Informatica                                                233
Ingenieria Aeroespacial                                    110
Ingenieria Electrica                                        81
Ingenieria Mecanica                                        141
Ingenieria Telematica                                      105
Ingenieria Termica y de Fluidos                            133
Ingenieria de Sistemas y Automatica                         83
Matematicas                                                 19
Mecanica de Medios Continuos y Teoria de Estructuras       100
Tecnologia Electronica                                      99
Teoria de la Senal y Comunicaciones 

In [ ]:
import os

output_folder = "TFE-Departamentos"
os.makedirs(output_folder, exist_ok=True)

for department, group_df in filtered_df.groupby("DEPARTAMENTO_TUTOR"):
    safe_filename = f"{department}.xlsx".replace("/", "_").replace(" ", "_")
    file_path = os.path.join(output_folder, safe_filename)
    group_df.to_excel(file_path, index=False, engine="openpyxl")
    print(f"Saved {safe_filename}")


Saved Departamento_de_Bioingenieria.xlsx
Saved Departamento_de_Ciencia_e_Ingenieria_de_Materiales_e_Ingenieria_Quimica.xlsx
Saved Departamento_de_Economia_de_la_Empresa.xlsx
Saved Departamento_de_Estadistica.xlsx
Saved Departamento_de_Fisica.xlsx
Saved Departamento_de_Informatica.xlsx
Saved Departamento_de_Ingenieria_Aeroespacial.xlsx
Saved Departamento_de_Ingenieria_Electrica.xlsx
Saved Departamento_de_Ingenieria_Mecanica.xlsx
Saved Departamento_de_Ingenieria_Telematica.xlsx
Saved Departamento_de_Ingenieria_Termica_y_de_Fluidos.xlsx
Saved Departamento_de_Ingenieria_de_Sistemas_y_Automatica.xlsx
Saved Departamento_de_Matematicas.xlsx
Saved Departamento_de_Mecanica_de_Medios_Continuos_y_Teoria_de_Estructuras.xlsx
Saved Departamento_de_Tecnologia_Electronica.xlsx
Saved Departamento_de_Teoria_de_la_Senal_y_Comunicaciones.xlsx
